<a href="https://colab.research.google.com/github/juanmmartinss/ProImg/blob/main/juanmmartinss/ProImg/proc_trab_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install rasterio

In [ ]:
import numpy as np
import os
import rasterio
import glob
from skimage.filters import threshold_otsu
import warnings
from skimage.transform import resize
from google.colab.patches import cv2_imshow
import cv2

# --- Configuração ---
PASTA_IMAGENS_PROCESSAR = '/content/drive/MyDrive/Imagens_PI_Final/'
PASTA_GROUND_TRUTH = '/content/drive/MyDrive/Imagens_Ground_Truth/'

dice_scores = []
warnings.filterwarnings('ignore', category=RuntimeWarning)

# Encontra imagens
lista_de_imagens = glob.glob(f"{PASTA_IMAGENS_PROCESSAR}/*_IMAGEM_*.tif")

if not lista_de_imagens:
    print(f"ERRO: Nenhuma imagem _IMAGEM_.tif encontrada em '{PASTA_IMAGENS_PROCESSAR}'")

for caminho_imagem in lista_de_imagens:
    try:
        nome_base_imagem = os.path.basename(caminho_imagem)
        print(f"\nProcessando Matéria-Prima: {nome_base_imagem}")

        # 1. PRÉ-PROCESSAMENTO
        with rasterio.open(caminho_imagem) as src:
            verde = src.read(1).astype(float)
            nir = src.read(2).astype(float)
            shape_10m = verde.shape

        # 2. NDWI
        ndwi = np.divide(
            verde - nir,
            verde + nir,
            out=np.zeros_like(verde),
            where=(verde + nir) != 0
        )
        ndwi_norm = cv2.normalize(ndwi, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)

        # 3. Otsu
        limiar = threshold_otsu(ndwi)
        resultado_obtido = (ndwi > limiar).astype(np.uint8)

        # 4. Ground Truth
        nome_arquivo_gabarito = nome_base_imagem.replace('_IMAGEM_', '_GABARITO_')
        caminho_gabarito = os.path.join(PASTA_GROUND_TRUTH, nome_arquivo_gabarito)

        with rasterio.open(caminho_gabarito) as gt_src:
            gabarito_30m = gt_src.read(1).astype(np.uint8)

        resultado_esperado = resize(
            gabarito_30m,
            shape_10m,
            order=0,
            preserve_range=True
        ).astype(np.uint8)

        # --- VISUALIZAÇÃO ---
        ndwi_gray = ndwi_norm
        otsu_mask = resultado_obtido * 255
        gt_mask = resultado_esperado * 255

        # Cria imagem colorida de comparação
        overlay = np.zeros((shape_10m[0], shape_10m[1], 3), dtype=np.uint8)
        overlay[(resultado_obtido == 1) & (resultado_esperado == 1)] = [0, 255, 0]   # ✅ Acerto (verde)
        overlay[(resultado_obtido == 1) & (resultado_esperado == 0)] = [0, 0, 255]   # ❌ Falso positivo (vermelho)
        overlay[(resultado_obtido == 0) & (resultado_esperado == 1)] = [255, 0, 0]   # ❌ Falso negativo (azul)

        # Monta uma vitrine horizontal com todas as imagens
        col1 = cv2.cvtColor(ndwi_gray, cv2.COLOR_GRAY2BGR)
        col2 = cv2.cvtColor(otsu_mask, cv2.COLOR_GRAY2BGR)
        col3 = cv2.cvtColor(gt_mask, cv2.COLOR_GRAY2BGR)
        visual = np.hstack([col1, col2, col3, overlay])

        print("Mostrando: NDWI | Otsu | Gabarito | Comparação (verde=acerto, azul=FN, vermelho=FP)")
        cv2_imshow(visual)

        # 5. DICE
        intersecao = np.sum((resultado_obtido == 1) & (resultado_esperado == 1))
        total_pixels = np.sum(resultado_obtido == 1) + np.sum(resultado_esperado == 1)
        dice_score = (2. * intersecao / total_pixels) if total_pixels > 0 else 1.0

        dice_scores.append(dice_score)
        print(f"  -> Score Dice (Dice Metric): {dice_score:.4f}")

    except Exception as e:
        print(f"  -> ERRO FATAL no processamento de {nome_base_imagem}: {e}")

# --- RESULTADO FINAL ---
if dice_scores:
    dice_medio = np.mean(dice_scores)
    print("\n--- RESULTADO FINAL ---")
    print(f"Imagens analisadas: {len(dice_scores)}")
    print(f"DICE MÉDIO FINAL: {dice_medio:.4f}")
else:
    print("\nNenhuma imagem foi processada com sucesso.")
